In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
train_dir = 'dataset/train'
test_dir = 'dataset/test'

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=20, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Create the training generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize all images to 150x150
    batch_size=32,
    class_mode='categorical'  # Because it's a multi-class classification
)

# Create the test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 3352 images belonging to 7 classes.
Found 813 images belonging to 7 classes.


# CNN Model

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import Input


In [16]:
model = Sequential()

model.add(Input(shape=(150, 150, 3)))
# Convolutional layer 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional layer 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional layer 3
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the layers
model.add(Flatten())

# Fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(train_generator.num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Model Training

In [18]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=25,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

Epoch 1/25
104/104 ━━━━━━━━━━━━━━━━━━━━ 86s 813ms/step - accuracy: 0.5792 - loss: 1.1765 - val_accuracy: 0.6050 - val_loss: 1.0939
Epoch 2/25
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 814us/step - accuracy: 0.6562 - loss: 0.9057 - val_accuracy: 0.6154 - val_loss: 0.9850
Epoch 3/25
104/104 ━━━━━━━━━━━━━━━━━━━━ 84s 791ms/step - accuracy: 0.5930 - loss: 1.1662 - val_accuracy: 0.6250 - val_loss: 1.0139
Epoch 4/25
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 845us/step - accuracy: 0.5938 - loss: 1.2003 - val_accuracy: 0.7692 - val_loss: 0.7191
Epoch 5/25
104/104 ━━━━━━━━━━━━━━━━━━━━ 91s 856ms/step - accuracy: 0.6005 - loss: 1.1383 - val_accuracy: 0.5063 - val_loss: 1.4717
Epoch 6/25
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 910us/step - accuracy: 0.5312 - loss: 1.3949 - val_accuracy: 0.6923 - val_loss: 0.7158
Epoch 7/25
104/104 ━━━━━━━━━━━━━━━━━━━━ 86s 805ms/step - accuracy: 0.5934 - loss: 1.1205 - val_accuracy: 0.6250 - val_loss: 1.0025
Epoch 8/25
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6562 - loss: 0.8848 

In [19]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.2f}")

26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 198ms/step - accuracy: 0.6911 - loss: 0.8220
Test accuracy: 0.70


In [20]:
model.save('car_classifier_model.h5')

In [21]:
model.save('car_classifier_model.keras')

# Predicting the result

In [25]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the saved model
model = tf.keras.models.load_model('car_classifier_model.h5')

# Load an image to predict
img_path = 'dataset/test/audi/604.jpg'
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # Reshape for model input

# Make the prediction
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction, axis=1)

# Get class labels
class_labels = list(train_generator.class_indices.keys())
print(f"Predicted class: {class_labels[predicted_class[0]]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
Predicted class: audi
